In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("gsk_YSslx8YUsEugSSbsXjGQWGdyb3FYdYgbbCYxVl6mHWtPSE7kcpTFyzi")

# Initialize LLM (Groq)
llm = ChatGroq(api_key=groq_api_key, model="llama3-8b-8192")

# Define State
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# Define Node
def chat_node(state: ChatState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

# Setup Checkpointer
checkpointer = InMemorySaver()

# Define Graph
graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

# Compile Chatbot
chatbot = graph.compile(checkpointer=checkpointer)